## **PLOTS**  
### *By Jyreneah Angel and Nicole Grace Joligon*  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from matplotlib.gridspec import GridSpec
from IPython.display import display 
from PIL import Image

## **BAR GRAPH**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Load dataset
bar_df = pd.read_csv("bar_assignment.csv")

# Transform data
bar_counts = bar_df.pivot_table(index='LABEL', columns='COUNT', aggfunc='size', fill_value=0)
bar_counts = bar_counts.rename(columns={0: 'No', 1: 'Yes'})["No", "Yes"].sort_index(ascending=False)

# Plot settings
fig, ax = plt.subplots(figsize=(20, 14))
bars = bar_counts.plot(kind='barh', stacked=True, ax=ax, color=['#FF6347', '#4682B4'])

# Add labels to bars
for container in bars.containers:
    bars.bar_label(container, labels=[int(v) if v > 0 else '' for v in container.datavalues], label_type='center')

# Formatting
ax.set_title('Bar Plot', fontweight='bold')
ax.set_xlabel('Count', fontweight='bold')
ax.set_ylabel('Labels', fontweight='bold')
ax.legend(title='Legend', labels=['No', 'Yes'])

plt.show()


## **Sankey Diagram**


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from matplotlib.gridspec import GridSpec
from IPython.display import display 
from PIL import Image

# Load dataset
df = pd.read_csv("sankey_assignment.csv")

source_columns = ['PS', 'OMP', 'CNP', 'NRP', 'NMCCC', 'PEC', 'NCDM', 'RGS']
target_columns = ['Reg', 'Aca', 'Oth']

# Initialize lists for Sankey diagram
sources, targets, values, link_colors = [], [], [], []

# Define colors for each label
colors_dict = {
    **dict.fromkeys(source_columns, '#FFA07A'),
    **dict.fromkeys(target_columns, '#3CB371'),
    'D': '#4782B4', 'F': '#5F9EA0', 'I': '#00BFFF', 'N': '#6395EC', 'S': '#87CEFA'
}

# Extract connections from source columns to LABEL
for source_col in source_columns:
    for _, row in df.iterrows():
        if row[source_col] > 0:
            sources.append(source_col)
            targets.append(row["LABEL"])
            values.append(row[source_col])
            link_colors.append(colors_dict.get(source_col, '#808080'))

# Extract connections from LABEL to target columns
for _, row in df.iterrows():
    for target_col in target_columns:
        if row[target_col] > 0:
            sources.append(row["LABEL"])
            targets.append(target_col)
            values.append(row[target_col])
            link_colors.append(colors_dict.get(row["LABEL"], '#808080'))

# Create label-to-index mapping
unique_labels = list(set(sources + targets))
label_indices = {label: i for i, label in enumerate(unique_labels)}

# Convert labels to indices
sources = [label_indices[src] for src in sources]
targets = [label_indices[tgt] for tgt in targets]

# Node colors
node_colors = [colors_dict.get(label, '#808080') for label in unique_labels]

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_labels,
        color=node_colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
))

fig.update_layout(title_text="Sankey Diagram of Flow", font_size=12, font_family="Arial")

# Save and display image
output_path = "./assets/sankey_diagram.png"
fig.write_image(output_path)
display(Image.open(output_path))

## **Network Graph**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib.gridspec import GridSpec
from IPython.display import display
from PIL import Image

# Load the CSV file
data = pd.read_csv("networks_assignment.csv")

# Create a graph
G = nx.Graph()
nodes = data.columns[1:].tolist()
G.add_nodes_from(nodes)

# Add edges based on the CSV data
for _, row in data.iterrows():
    node = row['LABELS']
    for target, value in row[1:].items():
        if value > 0:
            G.add_edge(node, target, weight=value)

# Define positions for pentagram nodes
pentagon_nodes = ['D', 'F', 'I', 'N', 'S']
angle = np.linspace(0, 2 * np.pi, len(pentagon_nodes), endpoint=False)
pos = {node: (np.cos(a), np.sin(a)) for node, a in zip(pentagon_nodes, angle)}

# Define positions for outer nodes
outer_radius = 2
yellow_nodes = ['AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'EST', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LUX', 'NLD', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP']
green_nodes = ['BIH', 'GEO', 'ISR', 'MNE', 'SRB', 'CHE', 'TUR', 'UKR', 'GBR', 'AUS', 'HKG', 'USA']
gray_nodes = ['ASU']
outer_nodes = green_nodes + yellow_nodes + gray_nodes
outer_angle = np.linspace(0, 2 * np.pi, len(outer_nodes), endpoint=False)
pos.update({node: (outer_radius * np.cos(a), outer_radius * np.sin(a)) for node, a in zip(outer_nodes, outer_angle)})

# Default position for unassigned nodes
for node in nodes:
    if node not in pos:
        pos[node] = (0, 0)

# Define node colors
color_map = {
    **dict.fromkeys(pentagon_nodes, 'blue'),
    **dict.fromkeys(green_nodes, 'green'),
    **dict.fromkeys(yellow_nodes, 'yellow'),
    'ASU': 'gray'
}
node_colors = [color_map.get(node, 'gray') for node in G.nodes()]

# Filter displayed nodes and edges
display_nodes = pentagon_nodes + outer_nodes
display_edges = [(u, v) for u, v, d in G.edges(data=True) if u in display_nodes and v in display_nodes]
H = G.edge_subgraph(display_edges).copy()

# Define edge colors
edge_colors = ['yellow' if v in yellow_nodes else 'green' if v in green_nodes else 'gray' if v in gray_nodes else 'blue' for u, v in H.edges()]

# Draw the graph
plt.figure()
nx.draw(H, {node: pos[node] for node in H.nodes()}, with_labels=True, node_color=[color_map[node] for node in H.nodes()], node_size=800, font_size=10, edge_color=edge_colors, width=2)
plt.title('Network Graph', fontweight='bold')
plt.show()